In [1]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 5.2 MB/s 
     |████████████████████████████████| 73 kB 1.1 MB/s 
     |████████████████████████████████| 57 kB 3.9 MB/s 
     |████████████████████████████████| 16.6 MB 23.5 MB/s 
     |████████████████████████████████| 1.2 MB 36.5 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


# **EXPERIMENT DIRECTION 2**

How does the amount of available data affect the learning of irregular past tense forms? Is there a threshold in the amount of data, or in the ratio between regular and irregular forms, that hinders the learning process?


In [161]:
import random

with open("english_merged_copy.txt",'r') as file:
    lines = file.readlines()
        
with open("reg.txt",'w') as file:
    for line in lines:
        # find() returns -1 if no match is found
        if line.find("irreg") == -1:
            file.write(line)

with open("irreg.txt",'w') as file:
    for line in lines:
        if line.find("irreg") != -1:
            file.write(line)

with open("reg.txt",'r') as file:
    lines_reg = file.readlines()

with open("irreg.txt",'r') as file:
    lines_irreg = file.readlines()

def create_dataset(lines_reg, lines_irreg, num_reg, num_irreg):
    with open("merged.txt",'w') as file:
        for i in range(num_reg):
            line_reg = random.choice(lines_reg)
            file.write(line_reg)
        for i in range(num_irreg):
            line_irreg = random.choice(lines_irreg)
            file.write(line_irreg)

In [162]:
create_dataset(lines_reg, lines_irreg,1120,140)

In [163]:
#create new set of src,trg,valid

# -*- coding: utf-8 -*-

import codecs
import _pickle as cPickle
import random
from collections import defaultdict
import pandas as pd

#set up output file
fout_src_train = codecs.open('experiment_1/src_train.txt','wb','utf-8')
fout_tgt_train = codecs.open('experiment_1/tgt_train.txt','wb','utf-8')
fout_src_valid = codecs.open('experiment_1/src_valid.txt','wb','utf-8')
fout_tgt_valid = codecs.open('experiment_1/tgt_valid.txt','wb','utf-8')
fout_src_test = codecs.open('experiment_1/src_test.txt','wb','utf-8')
fout_tgt_test = codecs.open('experiment_1/tgt_test.txt','wb','utf-8')

#read in  data
fin = codecs.open('merged.txt','rb','utf-8')

sources = []
targets = []

for line in fin:
	parts = line.strip().split()
	lemma = parts[2]
	form = parts[3]
	sources.append(' '.join(lemma))
	targets.append(' '.join(form))
fin.close()

pairs = list(zip(sources,targets))
random.shuffle(pairs)

#split into train and test
train = pairs[:int(.8*len(pairs))]
valid = pairs[int(.8*len(pairs)):int(.9*len(pairs))]
test = pairs[int(.9*len(pairs)):]

#write the outputs
for s,t in train:
	fout_src_train.write(s + '\n')
	fout_tgt_train.write(t + '\n')

for s,t in valid:
	fout_src_valid.write(s + '\n')
	fout_tgt_valid.write(t + '\n')

for s,t in test:
	fout_src_test.write(s + '\n')
	fout_tgt_test.write(t + '\n')



fout_src_train.close()
fout_tgt_train.close()
fout_src_valid.close()
fout_tgt_valid.close()
fout_src_test.close()
fout_tgt_test.close()


In [164]:
!onmt_build_vocab -config config-exp1.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-28 14:40:06,637 INFO] Counter vocab from 10000 samples.
[2022-03-28 14:40:06,637 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-03-28 14:40:06,653 INFO] corpus_1's transforms: TransformPipe()
[2022-03-28 14:40:06,757 INFO] Counters src:38
[2022-03-28 14:40:06,757 INFO] Counters tgt:38


In [165]:
!onmt_train -config config-exp1.yaml

[2022-03-28 14:40:12,760 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-28 14:40:12,760 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-28 14:40:12,761 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-28 14:40:12,761 INFO] Parsed 2 corpora from -data.
[2022-03-28 14:40:12,761 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-28 14:40:12,761 INFO] Loading vocab from text file...
[2022-03-28 14:40:12,761 INFO] Loading src vocabulary from experiment_1/run/example.vocab.src
[2022-03-28 14:40:12,761 INFO] Loaded src vocab has 38 tokens.
[2022-03-28 14:40:12,762 INFO] Loading tgt vocabulary from experiment_1/run/example.vocab.tgt
[2022-03-28 14:40:12,762 INFO] Loaded tgt vocab has 38 tokens.
[2022-03-28 14:40:12,762 INFO] Building fields with vocab in counters...
[2022-03-28 14:40:12,762 INFO]  * tgt vocab size: 42.
[2022-03-28 14:40:12,762 INFO]  * src vocab size

In [166]:
!onmt_translate -model experiment_1/run/model_step_1000.pt -src experiment_1/src_test.txt -output experiment_1/test-decoded.txt -gpu 0 -verbose -beam_size 12

[2022-03-28 14:42:26,909 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-03-28 14:42:27,010 INFO] 
SENT 1: ['r', '&', 'v', 'I', 'S']
PRED 1: r & v I S t
PRED SCORE: -0.0518

[2022-03-28 14:42:27,010 INFO] 
SENT 2: ['r', 'i', ':', 'g', '3', ':', 'd', 'Z', 'I', 't', 'e', 'I', 't']
PRED 2: r i : g 3 : d Z I t e I t I d
PRED SCORE: -0.7815

[2022-03-28 14:42:27,011 INFO] 
SENT 3: ['d', 'i', ':', 'v', '&', 'l', 'j', 'U']
PRED 3: d i : v & l j U d
PRED SCORE: -0.5528

[2022-03-28 14:42:27,011 INFO] 
SENT 4: ['r', '@

In [167]:
#For experimenting purpose

import os
os.remove("experiment_1/src_train.txt")
os.remove("experiment_1/src_test.txt")
os.remove("experiment_1/src_valid.txt")
os.remove("experiment_1/tgt_test.txt")
os.remove("experiment_1/tgt_train.txt")
os.remove("experiment_1/tgt_valid.txt")
os.remove("experiment_1/run/example.vocab.src")
os.remove("experiment_1/run/example.vocab.tgt")
os.remove("experiment_1/run/model_step_1000.pt")
os.remove("experiment_1/run/model_step_500.pt")